In [16]:
import ckanclient
from pprint import pprint
ckan = ckanclient.CkanClient('https://catalog.data.gov/api/3')
pprint(ckan.action('package_search', q='tags:energy', rows=5).keys())
ckan = ckan.action('package_search', q='tags:energy', rows=5)
print ckan['results'][4]

[u'count', u'sort', u'facets', u'results', u'search_facets']
{u'owner_org': u'6d90e48d-3076-4327-b2f4-c33919f374d5', u'maintainer': None, u'relationships_as_object': [], u'private': False, u'maintainer_email': None, u'num_tags': 0, u'id': u'159cd880-f4eb-49a2-bea0-5c719c490058', u'metadata_created': u'2016-09-24T15:28:53.130447', u'metadata_modified': u'2016-09-24T15:28:53.130447', u'author': None, u'author_email': None, u'state': u'active', u'version': None, u'license_id': None, u'type': u'dataset', u'resources': [{u'cache_last_updated': None, u'package_id': u'159cd880-f4eb-49a2-bea0-5c719c490058', u'webstore_last_updated': None, u'id': u'a0f2ace0-f3f3-47a6-993c-63a5b37a2db6', u'size': None, u'state': u'active', u'resource_locator_function': u'', u'hash': u'', u'description': u'FRS_Wastewater.zip', u'format': u'ZIP', u'tracking_summary': {u'total': 168, u'recent': 20}, u'mimetype_inner': None, u'url_type': None, u'resource_locator_protocol': u'', u'no_real_name': u'True', u'mimetype':

In [4]:
import ckanapi
ckan = ckanapi.RemoteCKAN('https://catalog.data.gov/api/3')
search_params = {                                           
    'q': 'tags:"energy"',  
    'fq': 'res_format:CSV',                                                                                   
}
d = ckan.call_action('package_search', data_dict=search_params) 
print d['count']

CKANAPIError: ['https://catalog.data.gov/api/3/api/action/package_search', 404, u'<html>\n <head>\n  <title>404 Not Found</title>\n </head>\n <body>\n  <h1>404 Not Found</h1>\n  The resource could not be found.<br /><br />\n\n\n\n </body>\n</html>']